In [121]:

import numpy as np

# --- Setup Parameters ---
S0 = 100.0   # Initial stock price
K = 70.0     # Strike Price
r = 0.05     # Risk-free rate
q = 0.10     # Dividend Yield (Crucial for early call exercise)
T_steps = 2  # Total periods (Max expiry is T+1 = 2)
dt = 1.0     # Time step (1 year)
sigma = 0.20 # Volatility (arbitrary for demonstration)

# Calculate Binomial Parameters (Risk-Neutral)
u = np.exp(sigma * np.sqrt(dt)) # Up factor
d = 1 / u                       # Down factor
R = np.exp(r * dt)              # Discount factor (Future Value of 1)
# Risk-Neutral Probability (adjusted for dividends q)
pu = (np.exp((r - q) * dt) - d) / (u - d)
pd = 1.0 - pu
discount = 1.0 / R

# --- 1. Option B: Expiry T+1 (2 periods) ---
# Maturity T+1 (k=2)
S_2 = np.array([S0 * u**2, S0 * u * d, S0 * d**2])
V_2B = np.maximum(S_2 - K, 0)

# Pre-Expiry T (k=1)
S_1 = np.array([S0 * u, S0 * d])
V_1B = np.zeros(2)

for j in range(2):
    # Continuation Value (CV) at k=1: Expected value of V_2B discounted
    CV = discount * (pu * V_2B[j] + pd * V_2B[j+1])
    # Exercise Value (EV) at k=1: Immediate exercise
    EV = np.maximum(S_1[j] - K, 0)
    # Option value at k=1 is max(EV, CV)
    V_1B[j] = max(EV, CV)

# Decision Day t (k=0)
S_0 = np.array([S0])
# Continuation Value (CV) at k=0
CV_0B = discount * (pu * V_1B[0] + pd * V_1B[1])
# Exercise Value (EV) at k=0
EV_0B = np.maximum(S_0[0] - K, 0)
V_0B = max(EV_0B, CV_0B)
optimal_B = "Exercise" if EV_0B > CV_0B else "Wait"


# --- 2. Option A: Expiry T (1 period) ---
# Maturity T (k=1)
S_1A = np.array([S0 * u, S0 * d])
V_1A = np.maximum(S_1A - K, 0)

# Decision Day t (k=0)
# Continuation Value (CV) at k=0
CV_0A = discount * (pu * V_1A[0] + pd * V_1A[1])
# Exercise Value (EV) at k=0
EV_0A = np.maximum(S_0[0] - K, 0)
V_0A = max(EV_0A, CV_0A)
optimal_A = "Exercise" if EV_0A > CV_0A else "Wait"

# --- Output ---
print(f"--- Option Parameters ---")
print(f"Stock Price (S0): ${S0:.2f}, Strike (K): ${K:.2f}, Dividend Yield (q): {q*100}%, Rate (r): {r*100}%")
print(f"Up Factor (u): {u:.4f}, Down Factor (d): {d:.4f}, R-N Prob (pu): {pu:.4f}")
print("="*40)
print(f"*** Decision at Day t ({S0:.2f}) ***")
print("="*40)

# Option B Results
print(f"Option B (Expiry T+1):")
print(f"  Intrinsic/Exercise Value (EV_0B): ${EV_0B:.4f}")
print(f"  Continuation Value (CV_0B):       ${CV_0B:.4f}")
print(f"  Optimal Action for B:             {optimal_B}")
print("-" * 40)

# Option A Results
print(f"Option A (Expiry T):")
print(f"  Intrinsic/Exercise Value (EV_0A): ${EV_0A:.4f}")
print(f"  Continuation Value (CV_0A):       ${CV_0A:.4f}")
print(f"  Optimal Action for A:             {optimal_A}")
print("="*40)

--- Option Parameters ---
Stock Price (S0): $100.00, Strike (K): $70.00, Dividend Yield (q): 10.0%, Rate (r): 5.0%
Up Factor (u): 1.2214, Down Factor (d): 0.8187, R-N Prob (pu): 0.3290
*** Decision at Day t (100.00) ***
Option B (Expiry T+1):
  Intrinsic/Exercise Value (EV_0B): $30.0000
  Continuation Value (CV_0B):       $23.8977
  Optimal Action for B:             Exercise
----------------------------------------
Option A (Expiry T):
  Intrinsic/Exercise Value (EV_0A): $30.0000
  Continuation Value (CV_0A):       $23.8977
  Optimal Action for A:             Exercise


In [122]:

import numpy as np
import pandas as pd

# --- Setup Parameters ---
X0 = 100.0   # Initial stock price
K = 105.0    # Strike Price (Cost to cover position)
T_periods = 3 # Total periods (Maturity)
beta = 1.0   # No discounting

# Random Variable Y Distribution (E[Y] = -1.00 < 0)
Y_up = 0.0
Y_down = -2.0
P_up = 0.5
P_down = 0.5


E_Y = P_up * Y_up + P_down * Y_down

# --- 1. Backward Induction Function ---
def value_put_option(price_tree, final_period):
    # Initialize the option value matrix
    # The matrix size is (T+1) rows by (T+1) columns, indexed (t, j)
    V = np.zeros((T_periods + 1, T_periods + 1))

    # Value at Maturity (t = T)
    t = T_periods
    V[t, :] = np.maximum(K - price_tree[t, :], 0)

    # Backward Induction Loop (from T-1 down to t=0)
    for t in range(T_periods - 1, -1, -1):
        for j in range(t + 1):
            current_price = price_tree[t, j]

            # 1. Exercise Value (EV): Payoff if exercised immediately
            EV = np.maximum(K - current_price, 0)

            # 2. Continuation Value (CV): Expected future value
            # The future state is X_t + Y

            # V_future_up: Value if Y = Y_up (Moves to state j in period t+1)
            V_future_up = V[t+1, j]

            # V_future_down: Value if Y = Y_down (Moves to state j+1 in period t+1)
            # The tree index mapping must reflect the path:
            # State j (t) -> State j (t+1) if Y_up
            # State j (t) -> State j+1 (t+1) if Y_down

            V_future_down = V[t+1, j+1]

            # Expected Value (E[V_{t+1}]) * beta (discount is 1)
            CV = beta * (P_up * V_future_up + P_down * V_future_down)

            # 3. Decision: Optimal value is max(EV, CV)
            V[t, j] = max(EV, CV)

            # --- Demonstration Check ---
            if EV > CV and t != T_periods: # Added t != T_periods check for clarity
                print(f"**Optimal Exercise Found!** at t={t}, price={current_price:.2f}. EV > CV: {EV:.4f} > {CV:.4f}")
                # The theorem proves this block should NOT be reached (or reached only at t=T)

    return V[0, 0], V

# --- 2. Construct the Stock Price Tree (Additive Model) ---
price_tree = np.zeros((T_periods + 1, T_periods + 1))
price_tree[0, 0] = X0

# Build the lattice forward (Price = Price_prev + Y)
for t in range(1, T_periods + 1):
    for j in range(t + 1):
        # State j in time t means j "down" moves have occurred.
        num_down_moves = j
        num_up_moves = t - j

        price_tree[t, j] = X0 + (num_down_moves * Y_down) + (num_up_moves * Y_up)

# --- 3. Run DP and Output ---
final_price, V_matrix = value_put_option(price_tree, T_periods)

print(f"--- Option Pricing DP Simulation ---")
print(f"Initial Price (X0): ${X0:.2f}, Strike (K): ${K:.2f}, E[Y]: {E_Y:.2f}")
print("="*60)
print(f"Stock Price Tree (X_t):")
masked_prices = np.where(price_tree == 0, np.nan, price_tree)
print(pd.DataFrame(masked_prices).to_markdown(floatfmt=".2f", numalign="right"))
print("="*60)

print(f"Option Value Matrix (V_t):")
# masked_values = np.where(V_matrix == 0, 0, V_matrix)
print(pd.DataFrame(V_matrix).to_markdown(floatfmt=".4f", numalign="right"))
print("="*60)
print(f"Final Conclusion: Value of American Put at t=0 is **${final_price:.4f}**")
print("No 'Optimal Exercise' messages printed before t=T, proving the theorem.")

--- Option Pricing DP Simulation ---
Initial Price (X0): $100.00, Strike (K): $105.00, E[Y]: -1.00
Stock Price Tree (X_t):
|    |      0 |     1 |     2 |     3 |
|---:|-------:|------:|------:|------:|
|  0 | 100.00 |   nan |   nan |   nan |
|  1 | 100.00 | 98.00 |   nan |   nan |
|  2 | 100.00 | 98.00 | 96.00 |   nan |
|  3 | 100.00 | 98.00 | 96.00 | 94.00 |
Option Value Matrix (V_t):
|    |      0 |      1 |       2 |       3 |
|---:|-------:|-------:|--------:|--------:|
|  0 | 8.0000 | 0.0000 |  0.0000 |  0.0000 |
|  1 | 7.0000 | 9.0000 |  0.0000 |  0.0000 |
|  2 | 6.0000 | 8.0000 | 10.0000 |  0.0000 |
|  3 | 5.0000 | 7.0000 |  9.0000 | 11.0000 |
Final Conclusion: Value of American Put at t=0 is **$8.0000**
No 'Optimal Exercise' messages printed before t=T, proving the theorem.


In [123]:

import numpy as np

# --- Input Parameters from Collateral Data  ---
Q = 100.0  # Initial Principal ($M)
r = 0.10   # Annual Interest Rate (10%)
k = 10     # Term (years)

# --- Formula Calculations ---

# 1. Calculate the Constant Annual Payment (P)
P_numerator = Q * r * (1 + r)**k
P_denominator = (1 + r)**k - 1
P_calculated = P_numerator / P_denominator

# 2. Calculate First Year's Interest (I1)
I1_calculated = Q * r

# 3. Calculate First Year's Scheduled Amortization (A1) from the payment P
A1_from_P = P_calculated - I1_calculated

# 4. Calculate First Year's Scheduled Amortization (A1) using the formula to be verified
A1_formula_numerator = Q * r
A1_formula_denominator = (1 + r)**k - 1
A1_from_Formula = A1_formula_numerator / A1_formula_denominator

# --- Verification Output ---

print(f"--- Exercise 15.1 Verification ---")
print(f"Parameters: Principal Q=${Q}M, Rate r={r*100}%, Term k={k} years")
print("-" * 45)

print(f"1. Constant Annual Payment (P): ${P_calculated:.4f} M")
print(f"2. First Year's Interest (I1 = Qr): ${I1_calculated:.4f} M")
print("-" * 45)

print(f"3. A1 derived from P: ${A1_from_P:.4f} M")
print(f"4. A1 derived from formula (Qr / ((1+r)^k - 1)): ${A1_from_Formula:.4f} M")

if np.isclose(A1_from_P, A1_from_Formula):
    print("\n✅ Verification Successful: Both calculated values for A1 match.")
else:
    print("\n❌ Verification Failed: Calculated values for A1 do not match.")

--- Exercise 15.1 Verification ---
Parameters: Principal Q=$100.0M, Rate r=10.0%, Term k=10 years
---------------------------------------------
1. Constant Annual Payment (P): $16.2745 M
2. First Year's Interest (I1 = Qr): $10.0000 M
---------------------------------------------
3. A1 derived from P: $6.2745 M
4. A1 derived from formula (Qr / ((1+r)^k - 1)): $6.2745 M

✅ Verification Successful: Both calculated values for A1 match.


In [124]:
import pandas as pd
import numpy as np

# --- Parameters ---
Q0 = 100.0        # Initial Principal ($M)
r = 0.10          # Annual Interest Rate (10%)
k = 10            # Original Mortgage Term (years)
periods = 10

# Calculate the constant total annual payment (P)
# P = Qr * (1+r)^k / ((1+r)^k - 1)
P_const = Q0 * r * (1 + r)**k / ((1 + r)**k - 1)

# Function to determine the Prepayment (PP) rate by year
def get_pp_rate(t):
    if t == 1:
        return 0.01  # 1%
    elif t == 2:
        return 0.03  # 3%
    else:
        return 0.06  # 6% for Year 3+

# --- Simulation ---
data_15_2 = []
Q_prev = Q0

for t in range(1, periods + 1):
    # Stop condition: principal is fully retired
    if Q_prev <= 1e-6 and t > 1:
        I_t, A_t, PP_t, P_t, Q_t, Q_minus_A = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    else:
        # 1. Interest & Scheduled Amortization
        I_t = r * Q_prev
        A_t = P_const - I_t

        # 2. Handle final principal payment (if amortization exceeds remaining principal)
        if Q_prev < A_t:
            A_t = Q_prev
            # Re-calculate constant payment based on the final principal payoff
            P_const_final = I_t + A_t

        Q_minus_A = max(0, Q_prev - A_t)

        # 3. Prepayment (PP)
        pp_rate = get_pp_rate(t) if t <= 8 else 0.0 # Stop prepayment if paid off early
        PP_t = pp_rate * Q_minus_A

        # 4. Total Principal and New Balance
        P_t = A_t + PP_t
        Q_t = max(0, Q_prev - P_t)

    data_15_2.append({
        'Year (t)': t,
        'Q_prev (Start)': Q_prev,
        'I_t (Interest)': I_t,
        'A_t (Sch. Amort.)': A_t,
        'PP_Rate': f"{pp_rate*100:.2f}%" if pp_rate > 0 else "0.00%",
        'PP_t (Prepay.)': PP_t,
        'P_t (Total Prin.)': P_t,
        'Q_t (End)': Q_t
    })

    Q_prev = Q_t

df_15_2 = pd.DataFrame(data_15_2)

# --- Formatting ---
cols_to_round = ['Q_prev (Start)', 'I_t (Interest)', 'A_t (Sch. Amort.)', 'PP_t (Prepay.)', 'P_t (Total Prin.)', 'Q_t (End)']
df_15_2[cols_to_round] = df_15_2[cols_to_round].round(4)
# Display the table
print("Constant Annual Payment (P): $", round(P_const, 4), "M")
print(df_15_2.to_markdown(index=False))

Constant Annual Payment (P): $ 16.2745 M
|   Year (t) |   Q_prev (Start) |   I_t (Interest) |   A_t (Sch. Amort.) | PP_Rate   |   PP_t (Prepay.) |   P_t (Total Prin.) |   Q_t (End) |
|-----------:|-----------------:|-----------------:|--------------------:|:----------|-----------------:|--------------------:|------------:|
|          1 |         100      |          10      |              6.2745 | 1.00%     |           0.9373 |              7.2118 |     92.7882 |
|          2 |          92.7882 |           9.2788 |              6.9957 | 3.00%     |           2.5738 |              9.5695 |     83.2187 |
|          3 |          83.2187 |           8.3219 |              7.9527 | 6.00%     |           4.516  |             12.4686 |     70.7501 |
|          4 |          70.7501 |           7.075  |              9.1995 | 6.00%     |           3.693  |             12.8926 |     57.8575 |
|          5 |          57.8575 |           5.7858 |             10.4888 | 6.00%     |           2.8421 |  

In [125]:

import pandas as pd
import numpy as np

# --- Parameters ---
EDR = 0.009  # Expected Default Rate (0.9%) [cite: 251]
WALs = list(range(1, 11))

# Loss Multiples by Credit Rating [cite: 251]
loss_multiples = {
    'AAA': 6.0,
    'AA': 5.0,
    'A': 4.0,
    'BBB': 3.0,
    'BB': 2.0,
    'B': 1.5,
    'CCC': 0.0
}

# --- Calculation ---
data_15_3 = []

for wal in WALs:
    row = {'WAL (Years)': wal}
    for rating, multiple in loss_multiples.items():
        # Required Buffer = WAL * EDR * Loss Multiple [cite: 251]
        buffer = wal * EDR * multiple
        row[rating] = buffer
    data_15_3.append(row)

df_15_3 = pd.DataFrame(data_15_3)

# --- Formatting (Convert to Percentage) ---
for rating in loss_multiples.keys():
    # Convert buffer to percentage and format as string
    df_15_3[rating] = (df_15_3[rating] * 100).round(2).astype(str) + '%'

# Display the table
print(df_15_3.to_markdown(index=False))

|   WAL (Years) | AAA   | AA    | A     | BBB   | BB    | B      | CCC   |
|--------------:|:------|:------|:------|:------|:------|:-------|:------|
|             1 | 5.4%  | 4.5%  | 3.6%  | 2.7%  | 1.8%  | 1.35%  | 0.0%  |
|             2 | 10.8% | 9.0%  | 7.2%  | 5.4%  | 3.6%  | 2.7%   | 0.0%  |
|             3 | 16.2% | 13.5% | 10.8% | 8.1%  | 5.4%  | 4.05%  | 0.0%  |
|             4 | 21.6% | 18.0% | 14.4% | 10.8% | 7.2%  | 5.4%   | 0.0%  |
|             5 | 27.0% | 22.5% | 18.0% | 13.5% | 9.0%  | 6.75%  | 0.0%  |
|             6 | 32.4% | 27.0% | 21.6% | 16.2% | 10.8% | 8.1%   | 0.0%  |
|             7 | 37.8% | 31.5% | 25.2% | 18.9% | 12.6% | 9.45%  | 0.0%  |
|             8 | 43.2% | 36.0% | 28.8% | 21.6% | 14.4% | 10.8%  | 0.0%  |
|             9 | 48.6% | 40.5% | 32.4% | 24.3% | 16.2% | 12.15% | 0.0%  |
|            10 | 54.0% | 45.0% | 36.0% | 27.0% | 18.0% | 13.5%  | 0.0%  |


In [126]:
import pandas as pd
import numpy as np

#  1. Generate Payment Schedule Data (Exercise 15.2) easier than reading csv file :)
Q0 = 100.0
r = 0.10
k = 10
periods = 10

P_const = Q0 * r * (1 + r)**k / ((1 + r)**k - 1)

def get_pp_rate(t):
    if t == 1: return 0.01
    elif t == 2: return 0.03
    else: return 0.06

payment_data = []
Q_prev = Q0

for t in range(1, periods + 1):
    if Q_prev <= 1e-6 and t > 1:
        P_t = 0.0
    else:
        I_t = r * Q_prev
        A_t = max(0, P_const - I_t)
        if Q_prev < A_t: A_t = Q_prev

        Q_minus_A = max(0, Q_prev - A_t)
        pp_rate = get_pp_rate(t) if t <= 8 else 0.0
        PP_t = pp_rate * Q_minus_A
        P_t = A_t + PP_t
        Q_t = max(0, Q_prev - P_t)
        Q_prev = Q_t

    payment_data.append({'Year': t, 'P_t': P_t})

df_payments = pd.DataFrame(payment_data)
P_values = df_payments['P_t'].values
max_maturity = len(P_values)

# --- 2. Calculate WAL Matrix
wal_matrix = np.full((max_maturity, max_maturity), np.nan)

def calculate_wal(j_start, t_end, P_array):
    """Calculates WAL for a tranche starting at year j_start and ending at year t_end."""
    years = np.arange(j_start, t_end + 1)
    P_sub_t = P_array[j_start-1:t_end]

    if np.isclose(np.sum(P_sub_t), 0): return 0.0

    wal = np.sum(years * P_sub_t) / np.sum(P_sub_t)
    return wal

for j_start in range(1, max_maturity + 1):
    for t_end in range(j_start, max_maturity + 1):
        wal = calculate_wal(j_start, t_end, P_values)
        wal_matrix[j_start-1, t_end-1] = wal

wal_df = pd.DataFrame(wal_matrix)
wal_df.columns = [f'End Year {t}' for t in range(1, max_maturity + 1)]
wal_df.index = [f'Start Year {j}' for j in range(1, max_maturity + 1)]

print("Tranche Weighted Average Life (WAL) Matrix:")
print(wal_df.iloc[:8, :8].to_markdown(floatfmt=".3f"))

Tranche Weighted Average Life (WAL) Matrix:
|              |   End Year 1 |   End Year 2 |   End Year 3 |   End Year 4 |   End Year 5 |   End Year 6 |   End Year 7 |   End Year 8 |
|:-------------|-------------:|-------------:|-------------:|-------------:|-------------:|-------------:|-------------:|-------------:|
| Start Year 1 |        1.000 |        1.570 |        2.180 |        2.737 |        3.281 |        3.822 |        4.364 |        4.910 |
| Start Year 2 |      nan     |        2.000 |        2.566 |        3.095 |        3.621 |        4.150 |        4.682 |        5.219 |
| Start Year 3 |      nan     |      nan     |        3.000 |        3.508 |        4.022 |        4.542 |        5.067 |        5.597 |
| Start Year 4 |      nan     |      nan     |      nan     |        4.000 |        4.508 |        5.022 |        5.542 |        6.067 |
| Start Year 5 |      nan     |      nan     |      nan     |      nan     |        5.000 |        5.508 |        6.022 |        6.542